In [1]:
"""
CHUNK 1: Estrutura Geral do Programa e Imports
Configuração inicial e carregamento do arquivo CSV principal
"""

import os
import pandas as pd
import numpy as np
from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage
from dotenv import load_dotenv
import time
import json
from datetime import datetime
import logging
import re

# Configurar logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('agrupamento_projetos.log'),
        logging.StreamHandler()
    ]
)

# Carregar variáveis de ambiente
load_dotenv()

# Carregar arquivo CSV principal
arquivo_dados = 'csv_longo/projetos_lei_do_bem_JUSTIFICATIVAS_RESULTADOS_PESSOAS.csv'
df = pd.read_csv(arquivo_dados, sep=';', encoding='utf-8')

print(f"📊 Arquivo carregado: {len(df)} registros encontrados")
print(f"📋 Colunas disponíveis: {len(df.columns)}")
print(f"📄 Arquivo: {arquivo_dados}")

# Definir colunas baseadas na estrutura do CSV carregado
colunas_identificacao = [
    'id_empresa_ano', 'empresa', 'ano_referencia'
]

colunas_analise = [
    'setor', 'natureza', 'tipo_pesquisa', 'projeto', 'projeto_resultados'
]

colunas_avaliacao = [
    'do_resultado_analise', 'p_resultado_analise'
]

# Configurações globais
LIMITE_PROJETOS_POR_LOTE = 50
TEMPO_PAUSA_ENTRE_REQUESTS = 2
MAX_TENTATIVAS = 3

# Verificar se as colunas necessárias existem
colunas_faltando = [col for col in colunas_analise if col not in df.columns]
if colunas_faltando:
    print(f"⚠️ Colunas não encontradas: {colunas_faltando}")
    print(f"📋 Colunas disponíveis no CSV: {list(df.columns)}")
else:
    print("✅ Todas as colunas necessárias encontradas")

print("✅ Chunk 1 executado: Imports, configurações e dados carregados")
print(f"📁 Colunas de identificação: {colunas_identificacao}")
print(f"🔍 Colunas para análise: {colunas_analise}")
print(f"📊 Colunas de avaliação: {colunas_avaliacao}")

📊 Arquivo carregado: 75348 registros encontrados
📋 Colunas disponíveis: 50
📄 Arquivo: csv_longo/projetos_lei_do_bem_JUSTIFICATIVAS_RESULTADOS_PESSOAS.csv
✅ Todas as colunas necessárias encontradas
✅ Chunk 1 executado: Imports, configurações e dados carregados
📁 Colunas de identificação: ['id_empresa_ano', 'empresa', 'ano_referencia']
🔍 Colunas para análise: ['setor', 'natureza', 'tipo_pesquisa', 'projeto', 'projeto_resultados']
📊 Colunas de avaliação: ['do_resultado_analise', 'p_resultado_analise']


/tmp/ipykernel_1204575/2852269188.py:33: DtypeWarning: Columns (25,26,32,34,35,36,39,41,43,46,48) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(arquivo_dados, sep=';', encoding='utf-8')


In [4]:
"""
CHUNK 2: Preparação dos Dados
Carrega o CSV e prepara os dados para processamento
"""

def extrair_dados_empresa_projeto(df):
    """
    Extrai CNPJ, Razão Social e Nome do Projeto das colunas concatenadas
    """
    try:
        df_temp = df.copy()
        
        # Extrair CNPJ da coluna 'empresa'
        df_temp['cnpj_extraido'] = df_temp['empresa'].str.extract(r'CNPJ:\s*([0-9/.]+)')
        
        # Extrair Razão Social da coluna 'empresa'
        df_temp['razao_social_extraida'] = df_temp['empresa'].str.extract(r'RAZÃO SOCIAL:\s*([^:]+?)(?:\s+ATIVIDADE ECONOMICA|$)')
        
        # Extrair Nome do Projeto da coluna 'projeto'
        df_temp['nome_projeto_extraido'] = df_temp['projeto'].str.extract(r'NOME:\s*([^:]+?)(?:\s+DESCRIÇÂO|$)')
        
        # Extrair indicador de ciclo multianual
        df_temp['ciclo_multianual'] = df_temp['projeto_multianual'].str.extract(r'CICLO MAIOR QUE 1 ANO:\s*([^:]+?)(?:\s+ATIVIDADE PDI|$)')
        
        # Limpar espaços em branco
        for col in ['cnpj_extraido', 'razao_social_extraida', 'nome_projeto_extraido', 'ciclo_multianual']:
            if col in df_temp.columns:
                df_temp[col] = df_temp[col].str.strip()
        
        logging.info(f"📊 Dados extraídos:")
        logging.info(f"   📋 CNPJs únicos: {df_temp['cnpj_extraido'].nunique()}")
        logging.info(f"   🏢 Razões sociais únicas: {df_temp['razao_social_extraida'].nunique()}")
        logging.info(f"   📄 Projetos multianuais: {len(df_temp[df_temp['ciclo_multianual'] == 'Sim'])}")
        
        return df_temp
    
    except Exception as e:
        logging.error(f"❌ Erro ao extrair dados: {e}")
        return df

def identificar_projetos_multianuais(df_temp):
    """
    Identifica grupos de projetos multianuais que devem ser ligados automaticamente
    """
    try:
        # Filtrar apenas projetos multianuais válidos
        projetos_multianuais = df_temp[
            (df_temp['ciclo_multianual'] == 'Sim') &
            (df_temp['cnpj_extraido'].notna()) &
            (df_temp['razao_social_extraida'].notna()) &
            (df_temp['nome_projeto_extraido'].notna())
        ].copy()
        
        if len(projetos_multianuais) == 0:
            logging.info("ℹ️ Nenhum projeto multianual encontrado")
            return []
        
        # Agrupar por CNPJ + Razão Social + Nome do Projeto
        grupos_multianuais = projetos_multianuais.groupby([
            'cnpj_extraido', 'razao_social_extraida', 'nome_projeto_extraido'
        ])
        
        grupos_identificados = []
        grupo_id_multianual = 1
        
        for (cnpj, razao, nome_proj), grupo in grupos_multianuais:
            if len(grupo) > 1:  # Apenas grupos com múltiplos anos
                anos_projeto = sorted(grupo['ano_referencia'].unique())
                indices_projeto = grupo.index.tolist()
                
                grupo_info = {
                    'grupo_id_multianual': f"MULTI_{grupo_id_multianual:04d}",
                    'cnpj': cnpj,
                    'razao_social': razao,
                    'nome_projeto': nome_proj,
                    'anos': anos_projeto,
                    'indices_df': indices_projeto,
                    'total_registros': len(grupo)
                }
                grupos_identificados.append(grupo_info)
                grupo_id_multianual += 1
        
        logging.info(f"🔗 Projetos multianuais identificados:")
        logging.info(f"   📊 Grupos multianuais: {len(grupos_identificados)}")
        logging.info(f"   📋 Total de registros multianuais: {sum(g['total_registros'] for g in grupos_identificados)}")
        
        # Mostrar alguns exemplos
        for i, grupo in enumerate(grupos_identificados[:3]):
            logging.info(f"   📄 Exemplo {i+1}: {grupo['nome_projeto'][:50]}... ({len(grupo['anos'])} anos: {grupo['anos']})")
        
        return grupos_identificados
    
    except Exception as e:
        logging.error(f"❌ Erro ao identificar projetos multianuais: {e}")
        return []

def aplicar_ligacao_automatica(df_temp, grupos_multianuais):
    """
    Aplica ligação automática aos projetos multianuais identificados
    """
    try:
        df_processado = df_temp.copy()
        df_processado['grupo_multianual'] = None
        df_processado['eh_multianual'] = False
        df_processado['anos_grupo_multianual'] = None
        
        total_projetos_ligados = 0
        
        for grupo in grupos_multianuais:
            grupo_id = grupo['grupo_id_multianual']
            indices = grupo['indices_df']
            anos_str = ', '.join(map(str, grupo['anos']))
            
            # Marcar todos os registros do grupo
            df_processado.loc[indices, 'grupo_multianual'] = grupo_id
            df_processado.loc[indices, 'eh_multianual'] = True
            df_processado.loc[indices, 'anos_grupo_multianual'] = anos_str
            
            total_projetos_ligados += len(indices)
        
        logging.info(f"✅ Ligação automática aplicada:")
        logging.info(f"   🔗 Projetos ligados automaticamente: {total_projetos_ligados}")
        logging.info(f"   📊 Grupos multianuais criados: {len(grupos_multianuais)}")
        
        return df_processado
    
    except Exception as e:
        logging.error(f"❌ Erro ao aplicar ligação automática: {e}")
        return df_temp

def preparar_dados_com_multianual(df):
    """
    Prepara dados identificando projetos multianuais e aplicando ligação automática
    """
    try:
        # Filtrar apenas registros com dados essenciais
        df_clean = df.dropna(subset=['projeto', 'setor', 'natureza', 'tipo_pesquisa'])
        df_clean = df_clean[df_clean['projeto'].str.len() > 50]
        
        logging.info(f"📊 Dados iniciais limpos: {len(df_clean)} registros")
        
        # Extrair dados estruturados
        df_temp = extrair_dados_empresa_projeto(df_clean)
        
        # Identificar projetos multianuais
        grupos_multianuais = identificar_projetos_multianuais(df_temp)
        
        # Aplicar ligação automática
        df_processado = aplicar_ligacao_automatica(df_temp, grupos_multianuais)
        
        # Preparar para agrupamento por LLM apenas dos projetos NÃO multianuais
        df_para_llm = df_processado[~df_processado['eh_multianual']].copy()
        
        # Obter combinações únicas (setor + tipo_pesquisa + natureza) dos projetos para LLM
        combinacoes = df_para_llm.groupby([
            'setor', 'tipo_pesquisa', 'natureza'
        ]).size().reset_index(name='count')
        
        # Filtrar combinações com pelo menos 3 projetos
        combinacoes_validas = combinacoes[combinacoes['count'] >= 3]
        
        # APLICAR FILTRO DE TESTE POR CATEGORIA (se ativo)
        if CATEGORIA_TESTE_API and isinstance(CATEGORIA_TESTE_API, str):
            combinacoes_original = len(combinacoes_validas)
            combinacoes_validas = combinacoes_validas[
                combinacoes_validas['setor'] == CATEGORIA_TESTE_API
            ]
            logging.info(f"🧪 FILTRO TESTE aplicado:")
            logging.info(f"   🎯 Categoria selecionada: '{CATEGORIA_TESTE_API}'")
            logging.info(f"   📊 Combinações antes do filtro: {combinacoes_original}")
            logging.info(f"   📊 Combinações após filtro: {len(combinacoes_validas)}")
            
            if len(combinacoes_validas) == 0:
                logging.warning(f"⚠️ Nenhuma combinação encontrada para categoria '{CATEGORIA_TESTE_API}'")
                categorias_disponiveis = df_para_llm['setor'].unique()[:10]  # Primeiras 10
                logging.info(f"📋 Categorias disponíveis (primeiras 10): {list(categorias_disponiveis)}")
        
        logging.info(f"📋 Resumo da preparação:")
        logging.info(f"   📊 Total de registros processados: {len(df_processado)}")
        logging.info(f"   🔗 Registros multianuais (já ligados): {len(df_processado[df_processado['eh_multianual']])}")
        logging.info(f"   🤖 Registros para LLM analisar: {len(df_para_llm)}")
        logging.info(f"   🏷️ Categorias para LLM processar: {len(combinacoes_validas)}")
        
        if len(combinacoes_validas) > 0:
            logging.info(f"   📊 Maior categoria para LLM: {combinacoes['count'].max()} projetos")
            logging.info(f"   📊 Média de projetos por categoria: {combinacoes['count'].mean():.1f}")
        
        return df_processado, df_para_llm, combinacoes_validas, grupos_multianuais
    
    except Exception as e:
        logging.error(f"❌ Erro na preparação com multianual: {e}")
        return None, None, None, None

def salvar_relatorio_multianuais(grupos_multianuais, timestamp):
    """
    Salva relatório detalhado dos projetos multianuais identificados
    """
    try:
        if not grupos_multianuais:
            return None
        
        # Criar DataFrame com informações dos grupos
        relatorio_data = []
        for grupo in grupos_multianuais:
            relatorio_data.append({
                'grupo_id_multianual': grupo['grupo_id_multianual'],
                'cnpj': grupo['cnpj'],
                'razao_social': grupo['razao_social'],
                'nome_projeto': grupo['nome_projeto'],
                'anos_projeto': ', '.join(map(str, grupo['anos'])),
                'total_anos': len(grupo['anos']),
                'total_registros': grupo['total_registros']
            })
        
        df_relatorio = pd.DataFrame(relatorio_data)
        arquivo_relatorio = f'resultados_agrupamento/projetos_multianuais_identificados_{timestamp}.csv'
        df_relatorio.to_csv(arquivo_relatorio, index=False, encoding='utf-8')
        
        logging.info(f"📄 Relatório de multianuais salvo: {arquivo_relatorio}")
        return arquivo_relatorio
    
    except Exception as e:
        logging.error(f"❌ Erro ao salvar relatório de multianuais: {e}")
        return None

# Configuração para teste limitado por categoria
CATEGORIA_TESTE_API = "Química e Farmácia"  # Mudar para False para processar todas as categorias
# CATEGORIA_TESTE_API = False  # Descomente esta linha para processar tudo

# Executar preparação com identificação de multianuais
print("\n🔄 Executando Chunk 2 ADAPTADO: Preparação com Ligação Automática de Projetos Multianuais")

if CATEGORIA_TESTE_API:
    print(f"🧪 MODO TESTE ATIVADO - Processando apenas categoria: '{CATEGORIA_TESTE_API}'")
else:
    print("🌐 MODO COMPLETO - Processando todas as categorias")

if 'df' in locals():
    # Timestamp para controle
    timestamp_preparacao = datetime.now().strftime('%Y%m%d_%H%M%S')
    
    # Processar dados com identificação de multianuais
    df_processado, df_para_llm, combinacoes_validas, grupos_multianuais = preparar_dados_com_multianual(df)
    
    if df_processado is not None:
        # Salvar relatório de multianuais
        arquivo_relatorio = salvar_relatorio_multianuais(grupos_multianuais, timestamp_preparacao)
        
        print(f"✅ Chunk 2 adaptado executado:")
        print(f"   📊 Total processado: {len(df_processado):,} registros")
        print(f"   🔗 Projetos multianuais ligados: {len(df_processado[df_processado['eh_multianual']]):,}")
        print(f"   🤖 Projetos para LLM: {len(df_para_llm):,}")
        print(f"   🏷️ Categorias para LLM: {len(combinacoes_validas)}")
        
        if CATEGORIA_TESTE_API:
            print(f"   🧪 MODO TESTE: Apenas categoria '{CATEGORIA_TESTE_API}'")
            if len(combinacoes_validas) > 0:
                total_projetos_teste = combinacoes_validas['count'].sum()
                print(f"   📊 Projetos a serem processados no teste: {total_projetos_teste}")
            else:
                print(f"   ⚠️ ATENÇÃO: Nenhuma combinação para categoria '{CATEGORIA_TESTE_API}'")
        
        print(f"   📄 Relatório salvo: {arquivo_relatorio}")
        
        # Estatísticas dos grupos multianuais
        if grupos_multianuais:
            total_grupos = len(grupos_multianuais)
            maior_grupo = max(g['total_registros'] for g in grupos_multianuais)
            print(f"   📈 Grupos multianuais: {total_grupos} (maior: {maior_grupo} registros)")
        
        # Renomear variáveis para compatibilidade com chunks seguintes
        df_clean = df_para_llm  # Para os chunks seguintes processarem apenas os não-multianuais
        
    else:
        print("❌ Chunk 2 adaptado falhou: Não foi possível processar os dados")
else:
    print("❌ DataFrame 'df' não encontrado. Execute Chunk 1 primeiro.")

2025-08-11 17:14:39,544 - INFO - 📊 Dados iniciais limpos: 74466 registros



🔄 Executando Chunk 2 ADAPTADO: Preparação com Ligação Automática de Projetos Multianuais
🧪 MODO TESTE ATIVADO - Processando apenas categoria: 'Química e Farmácia'


2025-08-11 17:14:40,213 - INFO - 📊 Dados extraídos:
2025-08-11 17:14:40,220 - INFO -    📋 CNPJs únicos: 5717
2025-08-11 17:14:40,222 - INFO -    🏢 Razões sociais únicas: 0
2025-08-11 17:14:40,269 - INFO -    📄 Projetos multianuais: 54529
2025-08-11 17:14:40,284 - INFO - ℹ️ Nenhum projeto multianual encontrado
2025-08-11 17:14:40,380 - INFO - ✅ Ligação automática aplicada:
2025-08-11 17:14:40,381 - INFO -    🔗 Projetos ligados automaticamente: 0
2025-08-11 17:14:40,382 - INFO -    📊 Grupos multianuais criados: 0
2025-08-11 17:14:40,548 - INFO - 🧪 FILTRO TESTE aplicado:
2025-08-11 17:14:40,550 - INFO -    🎯 Categoria selecionada: 'Química e Farmácia'
2025-08-11 17:14:40,550 - INFO -    📊 Combinações antes do filtro: 63
2025-08-11 17:14:40,551 - INFO -    📊 Combinações após filtro: 9
2025-08-11 17:14:40,552 - INFO - 📋 Resumo da preparação:
2025-08-11 17:14:40,552 - INFO -    📊 Total de registros processados: 74466
2025-08-11 17:14:40,554 - INFO -    🔗 Registros multianuais (já ligados): 0

✅ Chunk 2 adaptado executado:
   📊 Total processado: 74,466 registros
   🔗 Projetos multianuais ligados: 0
   🤖 Projetos para LLM: 74,466
   🏷️ Categorias para LLM: 9
   🧪 MODO TESTE: Apenas categoria 'Química e Farmácia'
   📊 Projetos a serem processados no teste: 14229
   📄 Relatório salvo: None


In [5]:
"""
CHUNK 3: Configuração da API Deepseek
Configura a conexão com a API Deepseek usando LangChain
"""

def configurar_api_deepseek():
    """
    Configura o cliente da API Deepseek
    """
    try:
        # Obter chave da API
        api_key = os.getenv('DEEPSEEK_API_KEY')
        if not api_key:
            raise ValueError("A chave da API do DeepSeek não está definida nas variáveis de ambiente.")
        
        # Configurar o modelo
        model = ChatOpenAI(
            model="deepseek-chat",
            temperature=0.3,  # Baixa temperatura para resultados mais consistentes
            base_url="https://api.deepseek.com",
            api_key=api_key,
            max_tokens=4000
        )
        
        logging.info("✅ API Deepseek configurada com sucesso")
        return model
    
    except Exception as e:
        logging.error(f"❌ Erro ao configurar API Deepseek: {e}")
        return None

def testar_conexao_api(model):
    """
    Testa a conexão com a API Deepseek
    """
    try:
        # Criar mensagem de teste simples
        mensagens_teste = [
            SystemMessage(content="Você é um assistente útil."),
            HumanMessage(content="Responda apenas 'OK' se você conseguir me ouvir.")
        ]
        
        # Fazer chamada de teste
        resposta = model.invoke(mensagens_teste)
        
        if resposta and resposta.content:
            logging.info("✅ Teste de conexão com API bem-sucedido")
            logging.info(f"📱 Resposta do teste: {resposta.content[:50]}...")
            return True
        else:
            logging.error("❌ Resposta vazia da API")
            return False
    
    except Exception as e:
        logging.error(f"❌ Erro no teste de conexão: {e}")
        return False

def estimar_custo_processamento(combinacoes_validas):
    """
    Estima o custo aproximado do processamento
    """
    if combinacoes_validas is None:
        return
    
    total_combinacoes = len(combinacoes_validas)
    total_projetos = combinacoes_validas['count'].sum()
    
    # Estimativas (valores aproximados para Deepseek)
    tokens_por_projeto = 300  # Média de tokens por projeto
    tokens_totais = total_projetos * tokens_por_projeto
    custo_por_1k_tokens = 0.0014  # USD por 1k tokens (aproximado Deepseek)
    custo_estimado = (tokens_totais / 1000) * custo_por_1k_tokens
    
    logging.info(f"💰 Estimativa de custo:")
    logging.info(f"   📊 Total de combinações: {total_combinacoes}")
    logging.info(f"   📋 Total de projetos: {total_projetos}")
    logging.info(f"   🔤 Tokens estimados: {tokens_totais:,}")
    logging.info(f"   💵 Custo estimado: ${custo_estimado:.2f} USD")
    
    return custo_estimado

# Executar configuração
print("\n🔄 Executando Chunk 3: Configuração da API Deepseek")

# Configurar API
model_deepseek = configurar_api_deepseek()

if model_deepseek:
    # Testar conexão
    conexao_ok = testar_conexao_api(model_deepseek)
    
    if conexao_ok and 'combinacoes_validas' in locals():
        # Estimar custo
        custo_estimado = estimar_custo_processamento(combinacoes_validas)
        
        print(f"✅ Chunk 3 executado: API configurada e testada")
        print(f"💰 Custo estimado: ${custo_estimado:.2f} USD" if custo_estimado else "Custo não calculado")
    else:
        print("⚠️ Chunk 3 parcial: API configurada mas teste falhou")
else:
    print("❌ Chunk 3 falhou: Não foi possível configurar a API")

2025-08-11 17:15:01,953 - INFO - ✅ API Deepseek configurada com sucesso



🔄 Executando Chunk 3: Configuração da API Deepseek


2025-08-11 17:15:02,528 - INFO - HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"
2025-08-11 17:15:05,273 - INFO - ✅ Teste de conexão com API bem-sucedido
2025-08-11 17:15:05,274 - INFO - 📱 Resposta do teste: OK...
2025-08-11 17:15:05,275 - INFO - 💰 Estimativa de custo:
2025-08-11 17:15:05,276 - INFO -    📊 Total de combinações: 9
2025-08-11 17:15:05,277 - INFO -    📋 Total de projetos: 14229
2025-08-11 17:15:05,278 - INFO -    🔤 Tokens estimados: 4,268,700
2025-08-11 17:15:05,279 - INFO -    💵 Custo estimado: $5.98 USD


✅ Chunk 3 executado: API configurada e testada
💰 Custo estimado: $5.98 USD


In [7]:
"""
CHUNK 4: Preparação do Template de Prompt para IA
Cria os templates de SystemMessage e HumanMessage para o Deepseek
"""

def criar_system_message():
    """
    Cria a mensagem do sistema com instruções para agrupamento
    """
    system_prompt = """Você é um especialista em análise de projetos de Pesquisa & Desenvolvimento (P&D) da Lei do Bem brasileira.

Sua tarefa é analisar projetos e agrupá-los por alta similaridade técnica e temática.

CRITÉRIOS DE AGRUPAMENTO:
1. Projetos devem ter ALTA SIMILARIDADE (>75%) em:
   - Objeto/tema principal do projeto
   - Tecnologias utilizadas
   - Metodologia aplicada
   - Resultados esperados

2. GRUPOS VÁLIDOS:
   - Mínimo: 2 projetos por grupo
   - Máximo: 8 projetos por grupo
   - Projetos únicos ficam sem grupo (grupo_id = 0)

3. CRITÉRIOS DE SIMILARIDADE:
   - Mesmo domínio tecnológico (ex: IoT, sensores, automação)
   - Mesma aplicação (ex: monitoramento, controle, otimização)
   - Metodologias similares (ex: machine learning, análise de dados)
   - Resultados comparáveis (ex: produtos, processos, softwares)

FORMATO DE SAÍDA:
Retorne APENAS um CSV com as colunas:
grupo_id,projeto_id,similaridade_score,justificativa_agrupamento

EXEMPLO:
1,ID123,0.85,"Ambos desenvolvem sensores IoT para automação industrial"
1,ID456,0.82,"Projetos focam em sensores para controle de processos"
2,ID789,0.90,"Desenvolvimento de algoritmos de machine learning"
2,ID012,0.88,"Aplicação de IA para análise preditiva"
0,ID345,0.00,"Projeto único sem similaridade suficiente"

IMPORTANTE:
- Seja rigoroso na similaridade
- Prefira menos grupos com alta qualidade
- Justifique cada agrupamento brevemente
- Analise todo o contexto do projeto, não apenas palavras-chave"""

    return SystemMessage(content=system_prompt)

def formatar_projetos_para_analise(df_subset):
    """
    Formata os projetos de um subset para análise pela IA
    """
    try:
        projetos_formatados = []
        
        for idx, row in df_subset.iterrows():
            # Extrair ID único do projeto da coluna 'projeto'
            projeto_texto = str(row['projeto'])
            
            # Buscar ID único entre ' ID ÚNICO: ' e ' NOME: '
            import re
            match_id = re.search(r'ID ÚNICO:\s*([^:]+?)\s+NOME:', projeto_texto)
            
            if match_id:
                projeto_id = match_id.group(1).strip()
            else:
                # Fallback caso não encontre o padrão
                projeto_id = f"PROJ_{row.get('id_empresa_ano', idx)}_{idx}"
                logging.warning(f"⚠️ ID único não encontrado para linha {idx}, usando fallback: {projeto_id}")
            
            # Formatação limpa do projeto
            projeto_formatado = f"""
ID: {projeto_id}
PROJETO: {row['projeto'][:500]}...
SETOR: {row['setor']}
NATUREZA: {row['natureza']}
TIPO: {row['tipo_pesquisa']}
RESULTADOS: {row['projeto_resultados'][:300] if pd.notna(row['projeto_resultados']) else 'Não informado'}...
"""
            projetos_formatados.append(projeto_formatado.strip())
        
        logging.info(f"📋 Formatados {len(projetos_formatados)} projetos para análise")
        return projetos_formatados
    
    except Exception as e:
        logging.error(f"❌ Erro ao formatar projetos: {e}")
        return []

def criar_human_message(projetos_formatados, combinacao_info):
    """
    Cria a mensagem humana com os projetos para análise
    """
    try:
        # Cabeçalho da análise
        cabecalho = f"""Analise os projetos abaixo e agrupe-os por alta similaridade técnica.

CONTEXTO DA ANÁLISE:
- Ano: {combinacao_info['ano_referencia']}
- Setor: {combinacao_info['setor']}
- Tipo de Pesquisa: {combinacao_info['tipo_pesquisa']}
- Natureza: {combinacao_info['natureza']}
- Total de Projetos: {len(projetos_formatados)}

PROJETOS PARA ANÁLISE:
{'='*50}"""

        # Adicionar projetos formatados
        projetos_texto = '\n\n'.join(projetos_formatados)
        
        # Instrução final
        instrucao_final = f"""
{'='*50}

Retorne APENAS o CSV com o agrupamento, seguindo o formato especificado no system prompt.
Analise cuidadosamente a similaridade técnica entre os projetos."""

        mensagem_completa = f"{cabecalho}\n\n{projetos_texto}\n{instrucao_final}"
        
        return HumanMessage(content=mensagem_completa)
    
    except Exception as e:
        logging.error(f"❌ Erro ao criar human message: {e}")
        return None

def validar_tamanho_prompt(system_msg, human_msg, limite_tokens=30000):
    """
    Valida se o prompt não excede o limite de tokens
    """
    try:
        # Estimativa simples: ~4 caracteres por token
        total_chars = len(system_msg.content) + len(human_msg.content)
        tokens_estimados = total_chars // 4
        
        if tokens_estimados > limite_tokens:
            logging.warning(f"⚠️ Prompt muito longo: {tokens_estimados} tokens estimados")
            return False
        
        logging.info(f"✅ Tamanho do prompt OK: {tokens_estimados} tokens estimados")
        return True
    
    except Exception as e:
        logging.error(f"❌ Erro ao validar tamanho do prompt: {e}")
        return False

# Executar preparação do template
print("\n🔄 Executando Chunk 4: Preparação do Template de Prompt")

# Criar system message
system_message_template = criar_system_message()

# Teste com dados dummy se disponível
if 'df_clean' in locals() and df_clean is not None and len(df_clean) > 0:
    # Pegar uma amostra pequena para teste
    df_teste = df_clean.head(3)
    projetos_teste = formatar_projetos_para_analise(df_teste)
    
    combinacao_teste = {
        'ano_referencia': df_teste.iloc[0]['ano_referencia'],
        'setor': df_teste.iloc[0]['setor'],
        'tipo_pesquisa': df_teste.iloc[0]['tipo_pesquisa'],
        'natureza': df_teste.iloc[0]['natureza']
    }
    
    human_message_teste = criar_human_message(projetos_teste, combinacao_teste)
    
    if human_message_teste:
        prompt_valido = validar_tamanho_prompt(system_message_template, human_message_teste)
        print(f"✅ Chunk 4 executado: Template criado e validado")
        print(f"📏 Prompt válido: {prompt_valido}")
    else:
        print("⚠️ Chunk 4 parcial: Template criado mas teste falhou")
else:
    print("✅ Chunk 4 executado: Template criado (sem teste - dados não disponíveis)")

2025-08-11 17:25:41,959 - INFO - 📋 Formatados 3 projetos para análise
2025-08-11 17:25:41,961 - INFO - ✅ Tamanho do prompt OK: 1180 tokens estimados



🔄 Executando Chunk 4: Preparação do Template de Prompt
✅ Chunk 4 executado: Template criado e validado
📏 Prompt válido: True


In [8]:
"""
CHUNK 5 ADAPTADO: Processamento por Categoria com Sub-lotes Inteligentes
Organiza processamento garantindo que todos os projetos de uma categoria sejam comparados
"""

# Novo limite baseado em tokens, não quantidade arbitrária
LIMITE_TOKENS_SEGUROS = 25000  # Deixa margem para resposta
TOKENS_POR_PROJETO = 350      # Estimativa conservadora
LIMITE_PROJETOS_POR_SUBLOTE = int(LIMITE_TOKENS_SEGUROS / TOKENS_POR_PROJETO)  # ~71 projetos

def dividir_categoria_em_sublotes(df_categoria, limite_projetos=LIMITE_PROJETOS_POR_SUBLOTE):
    """
    Divide uma categoria grande em sub-lotes, mantendo sobreposição para merge posterior
    """
    try:
        total_projetos = len(df_categoria)
        
        if total_projetos <= limite_projetos:
            # Categoria pequena: processar tudo de uma vez
            return [{
                'dados': df_categoria,
                'sublote_num': 1,
                'total_sublotes': 1,
                'tipo': 'categoria_completa',
                'sobreposicao': None
            }]
        
        # Categoria grande: dividir com sobreposição para merge posterior
        sublotes = []
        overlap_size = min(10, limite_projetos // 4)  # 25% de sobreposição, máximo 10
        
        inicio = 0
        sublote_num = 1
        
        while inicio < total_projetos:
            fim = min(inicio + limite_projetos, total_projetos)
            
            # Adicionar sobreposição (exceto no primeiro sub-lote)
            if sublote_num > 1:
                inicio_real = max(0, inicio - overlap_size)
            else:
                inicio_real = inicio
            
            sublote_dados = df_categoria.iloc[inicio_real:fim]
            
            sublote = {
                'dados': sublote_dados,
                'sublote_num': sublote_num,
                'total_sublotes': None,  # Será calculado depois
                'tipo': 'sublote_categoria',
                'sobreposicao': {
                    'inicio_original': inicio,
                    'fim_original': fim,
                    'overlap_inicio': overlap_size if sublote_num > 1 else 0,
                    'projetos_sobrepostos': overlap_size if sublote_num > 1 else 0
                }
            }
            sublotes.append(sublote)
            
            inicio = fim
            sublote_num += 1
        
        # Atualizar total de sub-lotes
        for sublote in sublotes:
            sublote['total_sublotes'] = len(sublotes)
        
        logging.info(f"📦 Categoria dividida: {total_projetos} projetos → {len(sublotes)} sub-lotes")
        logging.info(f"   🔗 Sobreposição: {overlap_size} projetos entre sub-lotes")
        
        return sublotes
    
    except Exception as e:
        logging.error(f"❌ Erro ao dividir categoria: {e}")
        return []

def filtrar_categoria_especifica(df_clean, combinacao):
    """
    Filtra TODOS os projetos de uma categoria específica (incluindo anos diferentes)
    """
    try:
        df_categoria = df_clean[
            (df_clean['setor'] == combinacao['setor']) &
            (df_clean['tipo_pesquisa'] == combinacao['tipo_pesquisa']) &
            (df_clean['natureza'] == combinacao['natureza'])
        ].copy()
        
        # Adicionar informação de ano para controle
        anos_encontrados = df_categoria['ano_referencia'].unique()
        
        logging.info(f"🔍 Categoria filtrada: {len(df_categoria)} projetos")
        logging.info(f"📅 Anos encontrados: {sorted(anos_encontrados)}")
        
        return df_categoria
    
    except Exception as e:
        logging.error(f"❌ Erro ao filtrar categoria: {e}")
        return pd.DataFrame()

def criar_nome_arquivo_categoria(combinacao, sublote_info=None):
    """
    Cria nome padronizado para arquivos por categoria
    """
    try:
        # Limpar caracteres especiais
        setor = re.sub(r'[^\w\s-]', '', str(combinacao['setor'])).strip()[:25]
        tipo = re.sub(r'[^\w\s-]', '', str(combinacao['tipo_pesquisa'])).strip()[:25]
        natureza = re.sub(r'[^\w\s-]', '', str(combinacao['natureza'])).strip()[:20]
        
        # Substituir espaços por underscores
        setor = setor.replace(' ', '_')
        tipo = tipo.replace(' ', '_')
        natureza = natureza.replace(' ', '_')
        
        if sublote_info and sublote_info['tipo'] == 'sublote_categoria':
            nome = f"grupos_categoria_{setor}_{tipo}_{natureza}_sublote{sublote_info['sublote_num']}.csv"
        else:
            nome = f"grupos_categoria_{setor}_{tipo}_{natureza}_completa.csv"
        
        return nome
    
    except Exception as e:
        logging.error(f"❌ Erro ao criar nome do arquivo: {e}")
        return f"grupos_categoria_erro_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"

def preparar_plano_processamento_categoria(df_clean, combinacoes_validas):
    """
    Prepara plano de processamento focado em categorias completas
    """
    try:
        plano_processamento = []
        
        for idx, row in combinacoes_validas.iterrows():
            combinacao = {
                'setor': row['setor'],
                'tipo_pesquisa': row['tipo_pesquisa'],
                'natureza': row['natureza'],
                'count': row['count']
            }
            
            # Filtrar TODOS os projetos da categoria
            df_categoria = filtrar_categoria_especifica(df_clean, combinacao)
            
            if len(df_categoria) == 0:
                continue
            
            # Dividir em sub-lotes se necessário
            sublotes = dividir_categoria_em_sublotes(df_categoria)
            
            for sublote in sublotes:
                plano_item = {
                    'combinacao': combinacao,
                    'dados': sublote['dados'],
                    'sublote_info': sublote,
                    'arquivo_saida': criar_nome_arquivo_categoria(combinacao, sublote),
                    'requer_merge': sublote['tipo'] == 'sublote_categoria'
                }
                plano_processamento.append(plano_item)
        
        logging.info(f"📋 Plano por categoria criado: {len(plano_processamento)} itens")
        
        # Estatísticas do plano
        categorias_completas = sum(1 for item in plano_processamento if not item['requer_merge'])
        categorias_sublotes = len(plano_processamento) - categorias_completas
        
        logging.info(f"   🟢 Categorias processadas completas: {categorias_completas}")
        logging.info(f"   🟡 Sub-lotes para merge posterior: {categorias_sublotes}")
        
        return plano_processamento
    
    except Exception as e:
        logging.error(f"❌ Erro ao preparar plano por categoria: {e}")
        return []

def salvar_plano_categoria(plano_processamento):
    """
    Salva o plano de processamento por categoria
    """
    try:
        plano_resumo = []
        for item in plano_processamento:
            resumo = {
                'setor': item['combinacao']['setor'],
                'tipo_pesquisa': item['combinacao']['tipo_pesquisa'],
                'natureza': item['combinacao']['natureza'],
                'total_projetos': len(item['dados']),
                'sublote_num': item['sublote_info']['sublote_num'],
                'total_sublotes': item['sublote_info']['total_sublotes'],
                'tipo_processamento': item['sublote_info']['tipo'],
                'requer_merge': item['requer_merge'],
                'arquivo_saida': item['arquivo_saida']
            }
            plano_resumo.append(resumo)
        
        df_plano = pd.DataFrame(plano_resumo)
        arquivo_plano = 'resultados_agrupamento/plano_processamento_categoria.csv'
        df_plano.to_csv(arquivo_plano, index=False, encoding='utf-8')
        
        logging.info(f"📄 Plano por categoria salvo: {arquivo_plano}")
        return arquivo_plano
    
    except Exception as e:
        logging.error(f"❌ Erro ao salvar plano categoria: {e}")
        return None

# Executar preparação por categoria
print("\n🔄 Executando Chunk 5 ADAPTADO: Processamento por Categoria")

if 'df_clean' in locals() and 'combinacoes_validas' in locals() and df_clean is not None:
    print(f"🎯 Nova estratégia: Categorias completas com sub-lotes inteligentes")
    print(f"📊 Limite por sub-lote: {LIMITE_PROJETOS_POR_SUBLOTE} projetos (~{LIMITE_TOKENS_SEGUROS:,} tokens)")
    
    # Criar plano por categoria
    plano_processamento = preparar_plano_processamento_categoria(df_clean, combinacoes_validas)
    
    if plano_processamento:
        # Salvar plano
        arquivo_plano = salvar_plano_categoria(plano_processamento)
        
        # Estatísticas finais
        total_projetos = sum(len(item['dados']) for item in plano_processamento)
        categorias_unicas = len(set((item['combinacao']['setor'], 
                                   item['combinacao']['tipo_pesquisa'], 
                                   item['combinacao']['natureza']) 
                                  for item in plano_processamento))
        
        print(f"✅ Chunk 5 adaptado executado:")
        print(f"   📊 Total de itens para processar: {len(plano_processamento)}")
        print(f"   🏷️ Categorias únicas: {categorias_unicas}")
        print(f"   📋 Total de projetos: {total_projetos}")
        print(f"   📄 Plano salvo: {arquivo_plano}")
        print(f"   🔗 Estratégia: Comparação completa dentro de cada categoria")
    else:
        print("❌ Chunk 5 adaptado falhou: Não foi possível criar plano de categoria")
else:
    print("⚠️ Chunk 5 adaptado ignorado: Dados não disponíveis dos chunks anteriores")

2025-08-11 17:26:00,263 - INFO - 🔍 Categoria filtrada: 1887 projetos
2025-08-11 17:26:00,264 - INFO - 📅 Anos encontrados: [np.float64(2018.0), np.float64(2019.0), np.float64(2020.0), np.float64(2021.0), np.float64(2022.0), np.float64(2023.0)]
2025-08-11 17:26:00,266 - INFO - 📦 Categoria dividida: 1887 projetos → 27 sub-lotes
2025-08-11 17:26:00,266 - INFO -    🔗 Sobreposição: 10 projetos entre sub-lotes
2025-08-11 17:26:00,304 - INFO - 🔍 Categoria filtrada: 8172 projetos
2025-08-11 17:26:00,305 - INFO - 📅 Anos encontrados: [np.float64(2018.0), np.float64(2019.0), np.float64(2020.0), np.float64(2021.0), np.float64(2022.0), np.float64(2023.0)]
2025-08-11 17:26:00,308 - INFO - 📦 Categoria dividida: 8172 projetos → 116 sub-lotes
2025-08-11 17:26:00,308 - INFO -    🔗 Sobreposição: 10 projetos entre sub-lotes
2025-08-11 17:26:00,326 - INFO - 🔍 Categoria filtrada: 165 projetos
2025-08-11 17:26:00,327 - INFO - 📅 Anos encontrados: [np.float64(2018.0), np.float64(2019.0), np.float64(2020.0), np.


🔄 Executando Chunk 5 ADAPTADO: Processamento por Categoria
🎯 Nova estratégia: Categorias completas com sub-lotes inteligentes
📊 Limite por sub-lote: 71 projetos (~25,000 tokens)


2025-08-11 17:26:00,425 - INFO - 🔍 Categoria filtrada: 575 projetos
2025-08-11 17:26:00,426 - INFO - 📅 Anos encontrados: [np.float64(2018.0), np.float64(2019.0), np.float64(2020.0), np.float64(2021.0), np.float64(2022.0), np.float64(2023.0)]
2025-08-11 17:26:00,427 - INFO - 📦 Categoria dividida: 575 projetos → 9 sub-lotes
2025-08-11 17:26:00,427 - INFO -    🔗 Sobreposição: 10 projetos entre sub-lotes
2025-08-11 17:26:00,444 - INFO - 🔍 Categoria filtrada: 42 projetos
2025-08-11 17:26:00,444 - INFO - 📅 Anos encontrados: [np.float64(2018.0), np.float64(2019.0), np.float64(2020.0), np.float64(2023.0)]
2025-08-11 17:26:00,445 - INFO - 📋 Plano por categoria criado: 206 itens
2025-08-11 17:26:00,446 - INFO -    🟢 Categorias processadas completas: 2
2025-08-11 17:26:00,446 - INFO -    🟡 Sub-lotes para merge posterior: 204
2025-08-11 17:26:00,450 - ERROR - ❌ Erro ao salvar plano categoria: Cannot save file into a non-existent directory: 'resultados_agrupamento'


✅ Chunk 5 adaptado executado:
   📊 Total de itens para processar: 206
   🏷️ Categorias únicas: 9
   📋 Total de projetos: 16199
   📄 Plano salvo: None
   🔗 Estratégia: Comparação completa dentro de cada categoria


In [12]:
"""
CHUNK 6 ADAPTADO: Loop Principal de Iteração com Requisições Assíncronas
Implementa processamento assíncrono para acelerar requisições à API
"""

import asyncio
import concurrent.futures
from functools import partial

def configurar_api_deepseek_async():
    """
    Configura o cliente da API Deepseek (usando ChatOpenAI normal)
    """
    try:
        # Obter chave da API
        api_key = os.getenv('DEEPSEEK_API_KEY')
        if not api_key:
            raise ValueError("A chave da API do DeepSeek não está definida nas variáveis de ambiente.")
        
        # Configurar o modelo (usar o ChatOpenAI normal)
        model = ChatOpenAI(
            model="deepseek-chat",
            temperature=0.3,
            base_url="https://api.deepseek.com",
            api_key=api_key,
            max_tokens=4000,
            max_retries=3,
            request_timeout=60.0
        )
        
        logging.info("✅ API Deepseek configurada para uso assíncrono")
        return model
    
    except Exception as e:
        logging.error(f"❌ Erro ao configurar API Deepseek: {e}")
        return None

async def processar_item_plano_async(item_plano, model_deepseek, system_message_template, executor, semaforo):
    """
    Processa um item individual do plano de processamento de forma assíncrona usando ThreadPoolExecutor
    """
    async with semaforo:  # Limitar número de requisições simultâneas
        try:
            combinacao = item_plano['combinacao']
            dados = item_plano['dados']
            arquivo_saida = item_plano['arquivo_saida']
            
            logging.info(f"🔄 Processando async: {combinacao['setor']} - {len(dados)} projetos")
            
            # Formatar projetos para análise
            projetos_formatados = formatar_projetos_para_analise(dados)
            
            if not projetos_formatados:
                logging.error("❌ Falha ao formatar projetos")
                return None
            
            # Criar mensagem humana
            human_message = criar_human_message(projetos_formatados, combinacao)
            
            if not human_message:
                logging.error("❌ Falha ao criar human message")
                return None
            
            # Validar tamanho do prompt
            if not validar_tamanho_prompt(system_message_template, human_message):
                logging.error("❌ Prompt excede limite de tokens")
                return None
            
            # Preparar mensagens para a API
            mensagens = [system_message_template, human_message]
            
            logging.info(f"📤 Enviando para API Deepseek (async via thread)...")
            
            # Executar chamada da API em thread separada para não bloquear
            loop = asyncio.get_event_loop()
            resposta = await loop.run_in_executor(
                executor, 
                partial(model_deepseek.invoke, mensagens)
            )
            
            if resposta and resposta.content:
                logging.info(f"✅ Resposta assíncrona recebida para {combinacao['setor']}")
                return {
                    'resposta': resposta.content,
                    'combinacao': combinacao,
                    'arquivo_saida': arquivo_saida,
                    'total_projetos': len(dados),
                    'requer_merge': item_plano.get('requer_merge', False)
                }
            else:
                logging.error("❌ Resposta vazia da API")
                return None
        
        except asyncio.TimeoutError:
            logging.error(f"⏱️ Timeout na requisição para {combinacao['setor']}")
            return None
        except Exception as e:
            logging.error(f"❌ Erro ao processar item async: {e}")
            return None

async def executar_lote_async(lote_itens, model_deepseek, system_message_template, executor, max_concurrent=3):
    """
    Executa um lote de itens de forma assíncrona
    """
    try:
        # Semáforo para limitar requisições simultâneas
        semaforo = asyncio.Semaphore(max_concurrent)
        
        # Criar tarefas assíncronas
        tarefas = []
        for item in lote_itens:
            tarefa = processar_item_plano_async(item, model_deepseek, system_message_template, executor, semaforo)
            tarefas.append(tarefa)
        
        # Executar todas as tarefas em paralelo
        resultados = await asyncio.gather(*tarefas, return_exceptions=True)
        
        # Filtrar resultados válidos
        resultados_validos = []
        for resultado in resultados:
            if isinstance(resultado, Exception):
                logging.error(f"❌ Exceção no processamento assíncrono: {resultado}")
            elif resultado is not None:
                resultados_validos.append(resultado)
        
        logging.info(f"✅ Lote assíncrono concluído: {len(resultados_validos)}/{len(lote_itens)} sucessos")
        return resultados_validos
    
    except Exception as e:
        logging.error(f"❌ Erro no lote assíncrono: {e}")
        return []

def dividir_em_lotes_async(plano_processamento, tamanho_lote=5):
    """
    Divide o plano em lotes menores para processamento assíncrono
    """
    try:
        lotes = []
        total_itens = len(plano_processamento)
        
        for i in range(0, total_itens, tamanho_lote):
            lote = plano_processamento[i:i+tamanho_lote]
            lotes.append(lote)
        
        logging.info(f"📦 Divisão para processamento assíncrono: {total_itens} itens → {len(lotes)} lotes")
        return lotes
    
    except Exception as e:
        logging.error(f"❌ Erro ao dividir em lotes async: {e}")
        return [plano_processamento]  # Retorna como um lote único

async def executar_loop_principal_async(plano_processamento, model_deepseek, system_message_template, 
                                       modo_teste=False, limite_teste=3, max_concurrent=3, tamanho_lote=5):
    """
    Executa o loop principal de processamento de forma assíncrona
    """
    try:
        resultados = []
        total_itens = len(plano_processamento)
        
        # Limitar para teste se necessário
        if modo_teste:
            plano_processamento = plano_processamento[:limite_teste]
            logging.info(f"🧪 Modo teste ativado: processando apenas {len(plano_processamento)} itens")
        
        logging.info(f"🚀 Iniciando processamento assíncrono de {len(plano_processamento)} itens...")
        logging.info(f"⚡ Configuração: {max_concurrent} requisições simultâneas, lotes de {tamanho_lote}")
        
        # Criar ThreadPoolExecutor para requisições HTTP
        with concurrent.futures.ThreadPoolExecutor(max_workers=max_concurrent) as executor:
            # Dividir em lotes para gerenciar melhor o processamento
            lotes = dividir_em_lotes_async(plano_processamento, tamanho_lote)
            
            for idx_lote, lote in enumerate(lotes, 1):
                try:
                    logging.info(f"\n{'='*60}")
                    logging.info(f"📦 Processando lote {idx_lote}/{len(lotes)} ({len(lote)} itens)")
                    
                    # Executar lote assíncrono
                    resultados_lote = await executar_lote_async(
                        lote, model_deepseek, system_message_template, executor, max_concurrent
                    )
                    
                    resultados.extend(resultados_lote)
                    
                    logging.info(f"✅ Lote {idx_lote} concluído: {len(resultados_lote)} sucessos")
                    
                    # Pausa entre lotes (exceto no último)
                    if idx_lote < len(lotes):
                        pausa_entre_lotes = min(TEMPO_PAUSA_ENTRE_REQUESTS, 3)  # Máximo 3s entre lotes
                        logging.info(f"⏸️ Pausando {pausa_entre_lotes}s entre lotes...")
                        await asyncio.sleep(pausa_entre_lotes)
                    
                except Exception as e:
                    logging.error(f"❌ Erro no lote {idx_lote}: {e}")
                    continue
        
        logging.info(f"\n🎉 Processamento assíncrono concluído!")
        logging.info(f"✅ Sucessos: {len(resultados)}/{total_itens}")
        
        return resultados
    
    except Exception as e:
        logging.error(f"❌ Erro no loop principal assíncrono: {e}")
        return []

def executar_processamento_sincronizado(plano_processamento, system_message_template, 
                                      modo_teste=True, limite_teste=3):
    """
    Função wrapper que gerencia a execução assíncrona usando ThreadPoolExecutor
    """
    try:
        # Configurar cliente (ChatOpenAI normal)
        model_deepseek = configurar_api_deepseek_async()
        if not model_deepseek:
            raise Exception("Falha na configuração da API")
        
        # Verificar se já há um loop de eventos rodando
        try:
            loop = asyncio.get_running_loop()
            logging.info("📡 Loop de eventos já ativo, usando nest_asyncio")
            
            # Se já há um loop, usar nest_asyncio
            try:
                import nest_asyncio
                nest_asyncio.apply()
            except ImportError:
                logging.warning("⚠️ nest_asyncio não instalado, tentando execução alternativa")
                # Fallback: executar de forma síncrona
                return executar_fallback_sincronizado(plano_processamento, model_deepseek, system_message_template, modo_teste, limite_teste)
            
            # Executar de forma assíncrona no loop existente
            resultado = asyncio.run(executar_loop_principal_async(
                plano_processamento, model_deepseek, system_message_template,
                modo_teste, limite_teste, max_concurrent=3, tamanho_lote=5
            ))
            
        except RuntimeError:
            # Não há loop rodando, criar um novo
            logging.info("📡 Criando novo loop de eventos")
            resultado = asyncio.run(executar_loop_principal_async(
                plano_processamento, model_deepseek, system_message_template,
                modo_teste, limite_teste, max_concurrent=3, tamanho_lote=5
            ))
        
        return resultado
    
    except Exception as e:
        logging.error(f"❌ Erro na execução sincronizada: {e}")
        # Fallback para processamento síncrono
        return executar_fallback_sincronizado(plano_processamento, model_deepseek, system_message_template, modo_teste, limite_teste)

def executar_fallback_sincronizado(plano_processamento, model_deepseek, system_message_template, modo_teste, limite_teste):
    """
    Fallback para processamento síncrono caso asyncio falhe
    """
    try:
        logging.info("🔄 Executando fallback síncrono...")
        resultados = []
        
        # Limitar para teste se necessário
        if modo_teste:
            plano_processamento = plano_processamento[:limite_teste]
        
        for idx, item in enumerate(plano_processamento, 1):
            try:
                logging.info(f"📋 Processando item {idx}/{len(plano_processamento)} (síncrono)")
                
                # Processar item de forma síncrona
                combinacao = item['combinacao']
                dados = item['dados']
                arquivo_saida = item['arquivo_saida']
                
                projetos_formatados = formatar_projetos_para_analise(dados)
                if not projetos_formatados:
                    continue
                
                human_message = criar_human_message(projetos_formatados, combinacao)
                if not human_message:
                    continue
                
                mensagens = [system_message_template, human_message]
                resposta = model_deepseek.invoke(mensagens)
                
                if resposta and resposta.content:
                    resultado = {
                        'resposta': resposta.content,
                        'combinacao': combinacao,
                        'arquivo_saida': arquivo_saida,
                        'total_projetos': len(dados),
                        'requer_merge': item.get('requer_merge', False)
                    }
                    resultados.append(resultado)
                    logging.info(f"✅ Item {idx} processado (síncrono)")
                
                # Pausa entre requisições
                if idx < len(plano_processamento):
                    time.sleep(TEMPO_PAUSA_ENTRE_REQUESTS)
                
            except Exception as e:
                logging.error(f"❌ Erro no item {idx}: {e}")
                continue
        
        return resultados
    
    except Exception as e:
        logging.error(f"❌ Erro no fallback síncrono: {e}")
        return []

def salvar_progresso_intermediario(resultados, timestamp):
    """
    Salva progresso intermediário durante o processamento
    """
    try:
        arquivo_progresso = f'resultados_agrupamento/progresso_{timestamp}.json'
        
        # Converter para formato serializável
        progresso_data = []
        for resultado in resultados:
            item = {
                'combinacao': resultado['combinacao'],
                'arquivo_saida': resultado['arquivo_saida'],
                'total_projetos': resultado['total_projetos'],
                'resposta_length': len(resultado['resposta']),
                'requer_merge': resultado.get('requer_merge', False),
                'timestamp': datetime.now().isoformat()
            }
            progresso_data.append(item)
        
        with open(arquivo_progresso, 'w', encoding='utf-8') as f:
            json.dump(progresso_data, f, indent=2, ensure_ascii=False)
        
        logging.info(f"💾 Progresso salvo: {arquivo_progresso}")
        return arquivo_progresso
    
    except Exception as e:
        logging.error(f"❌ Erro ao salvar progresso: {e}")
        return None

# Executar loop principal assíncrono
print("\n🔄 Executando Chunk 6 ADAPTADO: Loop Principal com Processamento Assíncrono")

if all(var in locals() for var in ['plano_processamento', 'system_message_template']):
    # Timestamp para controle
    timestamp_execucao = datetime.now().strftime('%Y%m%d_%H%M%S')
    
    print("⚡ PROCESSAMENTO ASSÍNCRONO ATIVADO")
    print("🧪 Executando em MODO TESTE (3 primeiros itens)")
    print("📊 Configuração: 3 requisições simultâneas, lotes de 5 itens")
    print("Para executar completo, mude modo_teste=False")
    
    # Executar processamento assíncrono
    resultados_processamento = executar_processamento_sincronizado(
        plano_processamento, 
        system_message_template,
        modo_teste=True,  # Mudar para False para execução completa
        limite_teste=3
    )
    
    if resultados_processamento:
        # Salvar progresso
        arquivo_progresso = salvar_progresso_intermediario(resultados_processamento, timestamp_execucao)
        
        print(f"✅ Chunk 6 adaptado executado: {len(resultados_processamento)} itens processados")
        print(f"⚡ Vantagem assíncrona: ~3x mais rápido que processamento sequencial")
        print(f"💾 Progresso salvo: {arquivo_progresso}")
    else:
        print("❌ Chunk 6 adaptado falhou: Nenhum item foi processado com sucesso")
else:
    print("⚠️ Chunk 6 adaptado ignorado: Dependências não disponíveis")
    missing_vars = [var for var in ['plano_processamento', 'system_message_template'] 
                    if var not in locals()]
    print(f"🔍 Variáveis faltando: {missing_vars}")

print("\n💡 DICA: Instale nest_asyncio se houver problemas com loops de eventos:")
print("pip install nest-asyncio")


🔄 Executando Chunk 6 ADAPTADO: Loop Principal com Processamento Assíncrono
⚠️ Chunk 6 adaptado ignorado: Dependências não disponíveis
🔍 Variáveis faltando: []

💡 DICA: Instale nest_asyncio se houver problemas com loops de eventos:
pip install nest-asyncio


In [ ]:
"""
CHUNK 7: Tratamento de Erros e Controle
Implementa retry logic, rate limiting e controle de erros
"""

def processar_item_com_retry(item_plano, model_deepseek, system_message_template, max_tentativas=MAX_TENTATIVAS):
    """
    Processa um item com lógica de retry em caso de erro
    """
    for tentativa in range(1, max_tentativas + 1):
        try:
            logging.info(f"🔄 Tentativa {tentativa}/{max_tentativas}")
            
            resultado = processar_item_plano(item_plano, model_deepseek, system_message_template)
            
            if resultado:
                if tentativa > 1:
                    logging.info(f"✅ Sucesso na tentativa {tentativa}")
                return resultado
            else:
                logging.warning(f"⚠️ Tentativa {tentativa} falhou - resultado vazio")
                
        except Exception as e:
            logging.error(f"❌ Tentativa {tentativa} falhou: {e}")
            
            # Pausa crescente entre tentativas (backoff exponencial)
            if tentativa < max_tentativas:
                pausa = TEMPO_PAUSA_ENTRE_REQUESTS * (2 ** (tentativa - 1))
                logging.info(f"⏸️ Pausando {pausa}s antes da próxima tentativa...")
                time.sleep(pausa)
    
    logging.error(f"❌ Todas as {max_tentativas} tentativas falharam")
    return None

def validar_resposta_api(resposta_content):
    """
    Valida se a resposta da API está no formato esperado
    """
    try:
        if not resposta_content:
            return False, "Resposta vazia"
        
        # Verificar se contém formato CSV básico
        linhas = resposta_content.strip().split('\n')
        
        if len(linhas) < 2:
            return False, "Resposta muito curta - esperado formato CSV"
        
        # Verificar se primeira linha parece um cabeçalho CSV
        primeira_linha = linhas[0].strip()
        if not any(campo in primeira_linha.lower() for campo in ['grupo', 'projeto', 'id']):
            return False, "Cabeçalho CSV não reconhecido"
        
        # Verificar se há pelo menos uma linha de dados
        segunda_linha = linhas[1].strip()
        if len(segunda_linha.split(',')) < 3:
            return False, "Formato de dados inválido"
        
        logging.info(f"✅ Resposta validada: {len(linhas)} linhas encontradas")
        return True, "Válida"
    
    except Exception as e:
        return False, f"Erro na validação: {e}"

def limpar_resposta_csv(resposta_content):
    """
    Limpa a resposta para extrair apenas o CSV válido
    """
    try:
        linhas = resposta_content.strip().split('\n')
        linhas_csv = []
        
        # Procurar início do CSV
        inicio_csv = -1
        for i, linha in enumerate(linhas):
            if any(campo in linha.lower() for campo in ['grupo_id', 'projeto_id', 'similaridade']):
                inicio_csv = i
                break
        
        if inicio_csv == -1:
            # Se não encontrar cabeçalho específico, usar primeira linha que parece CSV
            for i, linha in enumerate(linhas):
                if linha.count(',') >= 2:
                    inicio_csv = i
                    break
        
        if inicio_csv >= 0:
            for linha in linhas[inicio_csv:]:
                linha_limpa = linha.strip()
                if linha_limpa and not linha_limpa.startswith('#'):
                    linhas_csv.append(linha_limpa)
        
        csv_limpo = '\n'.join(linhas_csv)
        logging.info(f"🧹 CSV limpo: {len(linhas_csv)} linhas")
        return csv_limpo
    
    except Exception as e:
        logging.error(f"❌ Erro ao limpar CSV: {e}")
        return resposta_content

def salvar_erro_detalhado(item_plano, erro, timestamp):
    """
    Salva detalhes de erros para debug
    """
    try:
        arquivo_erro = f'resultados_agrupamento/logs/erro_{timestamp}.json'
        
        erro_data = {
            'timestamp': datetime.now().isoformat(),
            'combinacao': item_plano['combinacao'],
            'arquivo_saida': item_plano['arquivo_saida'],
            'total_projetos': len(item_plano['dados']),
            'erro': str(erro),
            'dados_amostra': {
                'primeiros_3_projetos': [
                    {
                        'setor': row['setor'],
                        'tipo_pesquisa': row['tipo_pesquisa'],
                        'projeto_preview': row['projeto'][:100] + '...' if len(row['projeto']) > 100 else row['projeto']
                    }
                    for _, row in item_plano['dados'].head(3).iterrows()
                ]
            }
        }
        
        with open(arquivo_erro, 'w', encoding='utf-8') as f:
            json.dump(erro_data, f, indent=2, ensure_ascii=False)
        
        logging.info(f"📋 Erro detalhado salvo: {arquivo_erro}")
        return arquivo_erro
    
    except Exception as e:
        logging.error(f"❌ Erro ao salvar detalhes do erro: {e}")
        return None

def monitorar_uso_api(resultados_processamento):
    """
    Monitora uso da API e custos aproximados
    """
    try:
        total_requisicoes = len(resultados_processamento)
        total_projetos = sum(r['total_projetos'] for r in resultados_processamento)
        
        # Estimativas de tokens e custo
        tokens_estimados = total_projetos * 300  # média por projeto
        custo_estimado = (tokens_estimados / 1000) * 0.0014  # preço aproximado Deepseek
        
        log_uso = {
            'timestamp': datetime.now().isoformat(),
            'total_requisicoes': total_requisicoes,
            'total_projetos': total_projetos,
            'tokens_estimados': tokens_estimados,
            'custo_estimado_usd': custo_estimado
        }
        
        arquivo_uso = 'resultados_agrupamento/logs/uso_api.json'
        with open(arquivo_uso, 'w', encoding='utf-8') as f:
            json.dump(log_uso, f, indent=2, ensure_ascii=False)
        
        logging.info(f"💰 Uso da API monitorado: {total_requisicoes} requests, ~${custo_estimado:.2f}")
        return log_uso
    
    except Exception as e:
        logging.error(f"❌ Erro ao monitorar uso da API: {e}")
        return None

# Executar tratamento de erros
print("\n🔄 Executando Chunk 7: Tratamento de Erros e Controle")

# Verificar se há resultados para validar
if 'resultados_processamento' in locals() and resultados_processamento:
    print(f"🔍 Validando {len(resultados_processamento)} resultados...")
    
    resultados_validos = []
    resultados_invalidos = []
    
    for resultado in resultados_processamento:
        valida, motivo = validar_resposta_api(resultado['resposta'])
        
        if valida:
            # Limpar CSV
            csv_limpo = limpar_resposta_csv(resultado['resposta'])
            resultado['resposta_limpa'] = csv_limpo
            resultados_validos.append(resultado)
        else:
            logging.warning(f"⚠️ Resposta inválida: {motivo}")
            resultados_invalidos.append(resultado)
    
    # Monitorar uso da API
    if resultados_validos:
        uso_api = monitorar_uso_api(resultados_validos)
    
    print(f"✅ Chunk 7 executado:")
    print(f"   ✅ Respostas válidas: {len(resultados_validos)}")
    print(f"   ❌ Respostas inválidas: {len(resultados_invalidos)}")
    
    if uso_api:
        print(f"   💰 Custo estimado: ${uso_api['custo_estimado_usd']:.2f}")
else:
    print("⚠️ Chunk 7 ignorado: Nenhum resultado para validar")
    resultados_validos = []

In [ ]:
"""
CHUNK 8: Consolidação Final integrando Multianuais + Merge de Sub-lotes
Une os grupos automáticos (multianuais) com os grupos da LLM e faz merge inteligente
"""

def identificar_categorias_para_merge(resultados_processados):
    """
    Identifica quais categorias foram divididas em sub-lotes e precisam de merge
    """
    try:
        categorias_sublotes = {}
        
        for resultado in resultados_processados:
            if resultado.get('requer_merge', False):
                combinacao = resultado['combinacao']
                categoria_key = f"{combinacao['setor']}_{combinacao['tipo_pesquisa']}_{combinacao['natureza']}"
                
                if categoria_key not in categorias_sublotes:
                    categorias_sublotes[categoria_key] = []
                
                categorias_sublotes[categoria_key].append(resultado)
        
        logging.info(f"🔗 Categorias que precisam merge: {len(categorias_sublotes)}")
        return categorias_sublotes
    
    except Exception as e:
        logging.error(f"❌ Erro ao identificar categorias para merge: {e}")
        return {}

def processar_merge_completo(resultados_processados):
    """
    Processa merge completo de todas as categorias que precisam
    """
    try:
        # Separar resultados que precisam de merge vs. categorias completas
        resultados_finais = []
        
        # Identificar categorias para merge
        categorias_sublotes = identificar_categorias_para_merge(resultados_processados)
        
        # Processar categorias que foram divididas em sub-lotes
        for categoria_key, sublotes in categorias_sublotes.items():
            resultado_merge = fazer_merge_sublotes_categoria(sublotes, categoria_key)
            if resultado_merge:
                resultados_finais.append(resultado_merge)
        
        # Adicionar categorias que foram processadas completas (sem sub-lotes)
        for resultado in resultados_processados:
            if not resultado.get('requer_merge', False):
                resultados_finais.append(resultado)
        
        logging.info(f"🔗 Merge completo finalizado:")
        logging.info(f"   🏷️ Categorias processadas: {len(resultados_finais)}")
        logging.info(f"   📦 Categorias que passaram por merge: {len(categorias_sublotes)}")
        
        return resultados_finais
    
    except Exception as e:
        logging.error(f"❌ Erro no merge completo: {e}")
        return resultados_processados  # Retorna originais em caso de erro

def fazer_merge_sublotes_categoria(sublotes_categoria, categoria_key):
    """
    Faz merge inteligente de todos os sub-lotes de uma categoria
    """
    try:
        logging.info(f"🔗 Fazendo merge da categoria: {categoria_key}")
        logging.info(f"   📦 Sub-lotes a processar: {len(sublotes_categoria)}")
        
        # Consolidar todos os DataFrames dos sub-lotes
        dfs_sublotes = []
        for sublote in sublotes_categoria:
            if 'dataframe' in sublote and sublote['dataframe'] is not None:
                df_temp = sublote['dataframe'].copy()
                df_temp['sublote_origem'] = sublote['arquivo_saida']
                dfs_sublotes.append(df_temp)
        
        if not dfs_sublotes:
            logging.error(f"❌ Nenhum DataFrame válido para categoria {categoria_key}")
            return None
        
        # Concatenar todos os sub-lotes
        df_categoria_completa = pd.concat(dfs_sublotes, ignore_index=True)
        
        # Renumerar grupos para evitar conflitos
        df_categoria_completa['grupo_id_original'] = df_categoria_completa['grupo_id']
        df_categoria_completa['grupo_id_global'] = 0
        
        # Lógica de merge simples: manter grupos únicos por sub-lote
        grupo_contador = 1
        
        for sublote_idx, sublote in enumerate(sublotes_categoria):
            df_sublote = dfs_sublotes[sublote_idx]
            grupos_sublote = df_sublote[df_sublote['grupo_id'] > 0]['grupo_id'].unique()
            
            for grupo_id in grupos_sublote:
                # Atribuir novo ID global
                mask = (df_categoria_completa['sublote_origem'] == sublote['arquivo_saida']) & \
                       (df_categoria_completa['grupo_id_original'] == grupo_id)
                df_categoria_completa.loc[mask, 'grupo_id_global'] = grupo_contador
                grupo_contador += 1
        
        # Remover duplicatas (projetos que apareceram em múltiplos sub-lotes)
        df_final = df_categoria_completa.drop_duplicates(subset=['projeto_id'], keep='first')
        
        # Limpar colunas auxiliares
        df_final = df_final.drop(['sublote_origem', 'grupo_id_original'], axis=1)
        
        grupos_finais = df_final[df_final['grupo_id_global'] > 0]['grupo_id_global'].nunique()
        projetos_agrupados = len(df_final[df_final['grupo_id_global'] > 0])
        
        logging.info(f"✅ Merge concluído para {categoria_key}:")
        logging.info(f"   📊 Projetos finais: {len(df_final)}")
        logging.info(f"   🏷️ Grupos após merge: {grupos_finais}")
        logging.info(f"   📈 Projetos agrupados: {projetos_agrupados}")
        
        return {
            'dataframe': df_final,
            'categoria': categoria_key,
            'grupos_finais': grupos_finais,
            'projetos_totais': len(df_final),
            'projetos_agrupados': projetos_agrupados,
            'sublotes_originais': len(sublotes_categoria)
        }
    
    except Exception as e:
        logging.error(f"❌ Erro no merge da categoria {categoria_key}: {e}")
        return None

def consolidar_resultados_completos(resultados_processados, df_processado, grupos_multianuais, timestamp):
    """
    Consolida TODOS os resultados: multianuais automáticos + grupos da LLM + merge de sub-lotes
    """
    try:
        logging.info("🔄 Iniciando consolidação completa...")
        
        # ETAPA 1: Processar merge dos sub-lotes (se necessário)
        if resultados_processados:
            resultados_merge = processar_merge_completo(resultados_processados)
            logging.info(f"✅ Merge de sub-lotes concluído: {len(resultados_merge)} categorias")
        else:
            resultados_merge = []
            logging.info("ℹ️ Nenhum resultado da LLM para fazer merge")
        
        # ETAPA 2: Consolidar resultados da LLM
        dataframes_llm = []
        if resultados_merge:
            for resultado in resultados_merge:
                if 'dataframe' in resultado and resultado['dataframe'] is not None:
                    df_temp = resultado['dataframe'].copy()
                    df_temp['origem_agrupamento'] = 'LLM'
                    df_temp['categoria_processamento'] = resultado.get('categoria', 'llm_categoria')
                    dataframes_llm.append(df_temp)
        
        # ETAPA 3: Criar DataFrame dos grupos multianuais automáticos
        df_multianuais_completo = df_processado[df_processado['eh_multianual']].copy()
        
        if len(df_multianuais_completo) > 0:
            # Mapear grupos multianuais para formato padrão
            df_grupos_multianuais = []
            
            for grupo in grupos_multianuais:
                grupo_id = grupo['grupo_id_multianual']
                indices = grupo['indices_df']
                
                # Buscar registros do grupo no df_processado
                registros_grupo = df_processado.loc[df_processado.index.isin(indices)]
                
                for idx, registro in registros_grupo.iterrows():
                    # Extrair ID único do projeto da coluna 'projeto'
                    projeto_texto = str(registro.get('projeto', ''))
                    
                    # Buscar ID único entre ' ID ÚNICO: ' e ' NOME: '
                    import re
                    match_id = re.search(r'ID ÚNICO:\s*([^:]+?)\s+NOME:', projeto_texto)
                    
                    if match_id:
                        projeto_id = match_id.group(1).strip()
                    else:
                        # Fallback caso não encontre o padrão
                        projeto_id = f"PROJ_{registro.get('id_empresa_ano', idx)}_{idx}"
                        logging.warning(f"⚠️ ID único não encontrado para registro multianual {idx}, usando fallback")
                    
                    grupo_row = {
                        'grupo_id_global': grupo_id,
                        'projeto_id': projeto_id,
                        'ano_referencia': registro['ano_referencia'],
                        'setor': registro['setor'],
                        'natureza': registro['natureza'],
                        'tipo_pesquisa': registro['tipo_pesquisa'],
                        'origem_agrupamento': 'MULTIANUAL_AUTO',
                        'categoria_processamento': f"multianual_{registro['setor']}_{registro['natureza']}",
                        'grupo_multianual_id': grupo_id,
                        'anos_grupo': registro['anos_grupo_multianual'],
                        'empresa': registro.get('empresa', ''),
                        'projeto': registro.get('projeto', '')
                    }
                    df_grupos_multianuais.append(grupo_row)
            
            df_multianuais_formatado = pd.DataFrame(df_grupos_multianuais)
            logging.info(f"✅ Grupos multianuais formatados: {len(df_multianuais_formatado)} registros")
        else:
            df_multianuais_formatado = pd.DataFrame()
            logging.info("ℹ️ Nenhum projeto multianual encontrado")
        
        # ETAPA 4: Consolidar tudo
        dataframes_finais = []
        
        # Adicionar resultados da LLM
        if dataframes_llm:
            dataframes_finais.extend(dataframes_llm)
            logging.info(f"✅ Adicionados {len(dataframes_llm)} DataFrames da LLM")
        
        # Adicionar grupos multianuais
        if len(df_multianuais_formatado) > 0:
            dataframes_finais.append(df_multianuais_formatado)
            logging.info(f"✅ Adicionados grupos multianuais")
        
        if not dataframes_finais:
            logging.error("❌ Nenhum resultado para consolidar")
            return None
        
        # Concatenar todos os resultados
        df_consolidado_final = pd.concat(dataframes_finais, ignore_index=True, sort=False)
        
        # ETAPA 5: Renumeração global dos grupos
        df_consolidado_final['grupo_id_final'] = 0
        contador_global = 1
        
        # Manter IDs multianuais como estão (já são únicos)
        mask_multianuais = df_consolidado_final['origem_agrupamento'] == 'MULTIANUAL_AUTO'
        grupos_multianuais_unicos = df_consolidado_final[mask_multianuais]['grupo_id_global'].unique()
        
        # Mapear grupos multianuais
        for grupo_multi in grupos_multianuais_unicos:
            mask_grupo = mask_multianuais & (df_consolidado_final['grupo_id_global'] == grupo_multi)
            df_consolidado_final.loc[mask_grupo, 'grupo_id_final'] = contador_global
            contador_global += 1
        
        # Mapear grupos da LLM por categoria
        mask_llm = df_consolidado_final['origem_agrupamento'] == 'LLM'
        if mask_llm.any():
            for categoria in df_consolidado_final[mask_llm]['categoria_processamento'].unique():
                mask_categoria = mask_llm & (df_consolidado_final['categoria_processamento'] == categoria)
                grupos_categoria = df_consolidado_final[mask_categoria & (df_consolidado_final['grupo_id_global'] > 0)]['grupo_id_global'].unique()
                
                for grupo_id in grupos_categoria:
                    mask_grupo = mask_categoria & (df_consolidado_final['grupo_id_global'] == grupo_id)
                    df_consolidado_final.loc[mask_grupo, 'grupo_id_final'] = contador_global
                    contador_global += 1
        
        # ETAPA 6: Gerar estatísticas consolidadas
        stats_consolidacao = {
            'timestamp': timestamp,
            'total_projetos': len(df_consolidado_final),
            'projetos_multianuais_auto': len(df_consolidado_final[df_consolidado_final['origem_agrupamento'] == 'MULTIANUAL_AUTO']),
            'projetos_llm': len(df_consolidado_final[df_consolidado_final['origem_agrupamento'] == 'LLM']),
            'total_grupos_finais': df_consolidado_final[df_consolidado_final['grupo_id_final'] > 0]['grupo_id_final'].nunique(),
            'projetos_agrupados_total': len(df_consolidado_final[df_consolidado_final['grupo_id_final'] > 0]),
            'projetos_isolados': len(df_consolidado_final[df_consolidado_final['grupo_id_final'] == 0]),
            'grupos_multianuais': len(grupos_multianuais_unicos) if mask_multianuais.any() else 0,
            'grupos_llm': len(df_consolidado_final[mask_llm & (df_consolidado_final['grupo_id_final'] > 0)]['grupo_id_final'].unique()) if mask_llm.any() else 0
        }
        
        stats_consolidacao['taxa_agrupamento_total'] = (stats_consolidacao['projetos_agrupados_total'] / stats_consolidacao['total_projetos']) * 100 if stats_consolidacao['total_projetos'] > 0 else 0
        
        logging.info(f"📊 Consolidação concluída:")
        logging.info(f"   📋 Total de projetos: {stats_consolidacao['total_projetos']:,}")
        logging.info(f"   🔗 Projetos multianuais: {stats_consolidacao['projetos_multianuais_auto']:,}")
        logging.info(f"   🤖 Projetos da LLM: {stats_consolidacao['projetos_llm']:,}")
        logging.info(f"   🏷️ Grupos finais: {stats_consolidacao['total_grupos_finais']}")
        logging.info(f"   📈 Taxa de agrupamento: {stats_consolidacao['taxa_agrupamento_total']:.1f}%")
        
        return {
            'dataframe_consolidado': df_consolidado_final,
            'estatisticas': stats_consolidacao
        }
    
    except Exception as e:
        logging.error(f"❌ Erro na consolidação completa: {e}")
        return None

def salvar_resultados_consolidados_finais(resultado_consolidacao, timestamp):
    """
    Salva todos os resultados consolidados finais
    """
    try:
        if not resultado_consolidacao:
            return None
        
        df_final = resultado_consolidacao['dataframe_consolidado']
        stats = resultado_consolidacao['estatisticas']
        
        # Arquivo principal consolidado
        arquivo_consolidado = f'resultados_agrupamento/GRUPOS_FINAL_COMPLETO_{timestamp}.csv'
        df_final.to_csv(arquivo_consolidado, index=False, encoding='utf-8')
        
        # Estatísticas detalhadas
        arquivo_stats = f'resultados_agrupamento/estatisticas_consolidacao_final_{timestamp}.json'
        with open(arquivo_stats, 'w', encoding='utf-8') as f:
            json.dump(stats, f, indent=2, ensure_ascii=False, default=str)
        
        # Relatório resumido por origem
        resumo_origem = df_final.groupby('origem_agrupamento').agg({
            'grupo_id_final': lambda x: (x > 0).sum(),  # projetos agrupados
            'projeto_id': 'count'  # total projetos
        }).reset_index()
        resumo_origem.columns = ['origem_agrupamento', 'projetos_agrupados', 'total_projetos']
        resumo_origem['taxa_agrupamento'] = (resumo_origem['projetos_agrupados'] / resumo_origem['total_projetos']) * 100
        
        arquivo_resumo = f'resultados_agrupamento/resumo_por_origem_{timestamp}.csv'
        resumo_origem.to_csv(arquivo_resumo, index=False, encoding='utf-8')
        
        # Relatório de grupos por tamanho
        distribuicao_grupos = df_final[df_final['grupo_id_final'] > 0].groupby('grupo_id_final').size().reset_index(name='tamanho_grupo')
        distribuicao_stats = distribuicao_grupos['tamanho_grupo'].describe()
        
        arquivo_distribuicao = f'resultados_agrupamento/distribuicao_grupos_{timestamp}.csv'
        distribuicao_grupos.to_csv(arquivo_distribuicao, index=False, encoding='utf-8')
        
        logging.info(f"💾 Resultados consolidados finais salvos:")
        logging.info(f"   📄 Arquivo principal: {arquivo_consolidado}")
        logging.info(f"   📊 Estatísticas: {arquivo_stats}")
        logging.info(f"   📋 Resumo por origem: {arquivo_resumo}")
        logging.info(f"   📈 Distribuição de grupos: {arquivo_distribuicao}")
        
        return {
            'arquivo_principal': arquivo_consolidado,
            'estatisticas': arquivo_stats,
            'resumo_origem': arquivo_resumo,
            'distribuicao_grupos': arquivo_distribuicao,
            'stats_distribuicao': distribuicao_stats.to_dict()
        }
    
    except Exception as e:
        logging.error(f"❌ Erro ao salvar resultados consolidados: {e}")
        return None

# Preparar Chunk 8 adaptado
print("\n🔄 Chunk 8 ADAPTADO: Consolidação Final com Multianuais + Merge")
print("📋 Este chunk integra:")
print("   🔗 Grupos multianuais (ligação automática)")
print("   🤖 Grupos da LLM (análise de similaridade)")
print("   📦 Merge de sub-lotes (unificação inteligente)")

def executar_consolidacao_completa(resultados_processados, df_processado, grupos_multianuais, timestamp):
    """
    Função principal para executar consolidação completa
    """
    if not any([resultados_processados, grupos_multianuais, df_processado is not None]):
        print("⚠️ Nenhum resultado disponível para consolidação")
        return None
    
    print(f"🔄 Iniciando consolidação completa...")
    
    # Consolidar todos os resultados
    resultado_consolidacao = consolidar_resultados_completos(
        resultados_processados, df_processado, grupos_multianuais, timestamp
    )
    
    if resultado_consolidacao:
        # Salvar resultados finais
        arquivos_salvos = salvar_resultados_consolidados_finais(resultado_consolidacao, timestamp)
        
        stats = resultado_consolidacao['estatisticas']
        print(f"✅ Consolidação completa concluída:")
        print(f"   📊 Total de projetos: {stats['total_projetos']:,}")
        print(f"   🔗 Multianuais automáticos: {stats['projetos_multianuais_auto']:,}")
        print(f"   🤖 Analisados por LLM: {stats['projetos_llm']:,}")
        print(f"   🏷️ Grupos finais: {stats['total_grupos_finais']}")
        print(f"   📈 Taxa de agrupamento total: {stats['taxa_agrupamento_total']:.1f}%")
        
        if arquivos_salvos:
            print(f"   📄 Arquivo final: {arquivos_salvos['arquivo_principal']}")
        
        return {
            'resultado_consolidacao': resultado_consolidacao,
            'arquivos_salvos': arquivos_salvos
        }
    else:
        print("❌ Falha na consolidação completa")
        return None

print("✅ Chunk 8 preparado - funções de consolidação completa carregadas")

In [ ]:
"""
CHUNK 9: Validação e Qualidade
Valida a qualidade dos agrupamentos e gera métricas de avaliação
"""

def analisar_qualidade_grupos(df_consolidado, df_clean):
    """
    Analisa a qualidade dos grupos formados
    """
    try:
        relatorio_qualidade = {}
        
        # 1. Análise de distribuição dos grupos
        grupos_validos = df_consolidado[df_consolidado['grupo_id_global'] > 0]
        distribuicao_tamanhos = grupos_validos.groupby('grupo_id_global').size()
        
        relatorio_qualidade['distribuicao_grupos'] = {
            'total_grupos': len(distribuicao_tamanhos),
            'tamanho_medio': distribuicao_tamanhos.mean(),
            'tamanho_mediano': distribuicao_tamanhos.median(),
            'maior_grupo': distribuicao_tamanhos.max(),
            'menor_grupo': distribuicao_tamanhos.min(),
            'grupos_por_tamanho': distribuicao_tamanhos.value_counts().to_dict()
        }
        
        # 2. Análise por setor/categoria
        analise_categorias = []
        for (setor, natureza, tipo), grupo_cat in df_consolidado.groupby(['setor', 'natureza', 'tipo_pesquisa']):
            grupos_formados = grupo_cat[grupo_cat['grupo_id_global'] > 0]['grupo_id_global'].nunique()
            taxa_agrupamento = len(grupo_cat[grupo_cat['grupo_id_global'] > 0]) / len(grupo_cat) * 100
            
            analise_cat = {
                'setor': setor,
                'natureza': natureza,
                'tipo_pesquisa': tipo,
                'total_projetos': len(grupo_cat),
                'grupos_formados': grupos_formados,
                'projetos_agrupados': len(grupo_cat[grupo_cat['grupo_id_global'] > 0]),
                'taxa_agrupamento': taxa_agrupamento,
                'eficiencia_agrupamento': grupos_formados / len(grupo_cat) if len(grupo_cat) > 0 else 0
            }
            analise_categorias.append(analise_cat)
        
        relatorio_qualidade['analise_por_categoria'] = analise_categorias
        
        # 3. Detecção de possíveis problemas
        problemas_detectados = []
        
        # Grupos muito grandes (possível super-agrupamento)
        grupos_grandes = distribuicao_tamanhos[distribuicao_tamanhos > 10]
        if len(grupos_grandes) > 0:
            problemas_detectados.append({
                'tipo': 'super_agrupamento',
                'descricao': f'{len(grupos_grandes)} grupos com mais de 10 projetos',
                'grupos_afetados': grupos_grandes.index.tolist()
            })
        
        # Taxa de agrupamento muito baixa por categoria
        categorias_baixa_taxa = [cat for cat in analise_categorias if cat['taxa_agrupamento'] < 20]
        if categorias_baixa_taxa:
            problemas_detectados.append({
                'tipo': 'baixa_taxa_agrupamento',
                'descricao': f'{len(categorias_baixa_taxa)} categorias com taxa < 20%',
                'categorias_afetadas': categorias_baixa_taxa
            })
        
        relatorio_qualidade['problemas_detectados'] = problemas_detectados
        
        logging.info(f"📊 Qualidade analisada: {len(distribuicao_tamanhos)} grupos válidos")
        logging.info(f"⚠️ Problemas detectados: {len(problemas_detectados)}")
        
        return relatorio_qualidade
    
    except Exception as e:
        logging.error(f"❌ Erro na análise de qualidade: {e}")
        return None

def gerar_amostras_grupos(df_consolidado, df_clean, num_amostras=5):
    """
    Gera amostras de grupos para validação manual
    """
    try:
        amostras = []
        grupos_validos = df_consolidado[df_consolidado['grupo_id_global'] > 0]
        
        # Selecionar grupos de diferentes tamanhos
        distribuicao_tamanhos = grupos_validos.groupby('grupo_id_global').size()
        grupos_pequenos = distribuicao_tamanhos[distribuicao_tamanhos == 2].head(2).index
        grupos_medios = distribuicao_tamanhos[(distribuicao_tamanhos >= 3) & (distribuicao_tamanhos <= 5)].head(2).index
        grupos_grandes = distribuicao_tamanhos[distribuicao_tamanhos > 5].head(1).index
        
        grupos_amostra = list(grupos_pequenos) + list(grupos_medios) + list(grupos_grandes)
        
        for grupo_id in grupos_amostra[:num_amostras]:
            projetos_grupo = grupos_validos[grupos_validos['grupo_id_global'] == grupo_id]
            
            # Buscar detalhes dos projetos no DataFrame original
            detalhes_projetos = []
            for _, projeto in projetos_grupo.iterrows():
                # Tentar encontrar projeto correspondente no df_clean
                projeto_id = projeto['projeto_id']
                
                # Buscar no df_clean (pode precisar ajustar a lógica de matching)
                projeto_detalhes = {
                    'projeto_id': projeto_id,
                    'setor': projeto['setor'],
                    'natureza': projeto['natureza'],
                    'tipo_pesquisa': projeto['tipo_pesquisa'],
                    'ano': projeto['ano_referencia']
                }
                
                # Tentar buscar projeto completo (simplificado para exemplo)
                try:
                    # Aqui você pode implementar lógica mais sofisticada de matching
                    matches = df_clean[
                        (df_clean['setor'] == projeto['setor']) &
                        (df_clean['ano_referencia'] == projeto['ano_referencia'])
                    ]
                    if len(matches) > 0:
                        primeiro_match = matches.iloc[0]
                        projeto_detalhes['projeto_preview'] = primeiro_match['projeto'][:200] + '...'
                        projeto_detalhes['resultados_preview'] = str(primeiro_match.get('projeto_resultados', ''))[:150] + '...'
                except:
                    projeto_detalhes['projeto_preview'] = 'Detalhes não encontrados'
                    projeto_detalhes['resultados_preview'] = 'N/A'
                
                detalhes_projetos.append(projeto_detalhes)
            
            amostra = {
                'grupo_id': int(grupo_id),
                'tamanho_grupo': len(projetos_grupo),
                'combinacao': {
                    'setor': projetos_grupo.iloc[0]['setor'],
                    'natureza': projetos_grupo.iloc[0]['natureza'],
                    'tipo_pesquisa': projetos_grupo.iloc[0]['tipo_pesquisa'],
                    'ano': projetos_grupo.iloc[0]['ano_referencia']
                },
                'projetos': detalhes_projetos
            }
            amostras.append(amostra)
        
        logging.info(f"📋 Geradas {len(amostras)} amostras para validação")
        return amostras
    
    except Exception as e:
        logging.error(f"❌ Erro ao gerar amostras: {e}")
        return []

def calcular_metricas_agrupamento(df_consolidado):
    """
    Calcula métricas quantitativas do agrupamento
    """
    try:
        metricas = {}
        
        total_projetos = len(df_consolidado)
        projetos_agrupados = len(df_consolidado[df_consolidado['grupo_id_global'] > 0])
        total_grupos = df_consolidado[df_consolidado['grupo_id_global'] > 0]['grupo_id_global'].nunique()
        
        # Métricas básicas
        metricas['cobertura'] = projetos_agrupados / total_projetos if total_projetos > 0 else 0
        metricas['densidade_media'] = projetos_agrupados / total_grupos if total_grupos > 0 else 0
        metricas['eficiencia'] = total_grupos / total_projetos if total_projetos > 0 else 0
        
        # Distribuição por tamanho
        distribuicao = df_consolidado[df_consolidado['grupo_id_global'] > 0].groupby('grupo_id_global').size()
        metricas['coeficiente_variacao_tamanho'] = distribuicao.std() / distribuicao.mean() if distribuicao.mean() > 0 else 0
        
        # Métricas por categoria
        metricas_por_categoria = {}
        for (setor, natureza), grupo_cat in df_consolidado.groupby(['setor', 'natureza']):
            cat_key = f"{setor}_{natureza}"
            grupos_cat = grupo_cat[grupo_cat['grupo_id_global'] > 0]['grupo_id_global'].nunique()
            projetos_cat = len(grupo_cat)
            
            metricas_por_categoria[cat_key] = {
                'total_projetos': projetos_cat,
                'grupos_formados': grupos_cat,
                'taxa_agrupamento': len(grupo_cat[grupo_cat['grupo_id_global'] > 0]) / projetos_cat if projetos_cat > 0 else 0,
                'produtividade': grupos_cat / projetos_cat if projetos_cat > 0 else 0
            }
        
        metricas['por_categoria'] = metricas_por_categoria
        
        logging.info(f"📊 Métricas calculadas:")
        logging.info(f"   📈 Cobertura: {metricas['cobertura']:.2%}")
        logging.info(f"   🎯 Densidade média: {metricas['densidade_media']:.1f} projetos/grupo")
        logging.info(f"   ⚡ Eficiência: {metricas['eficiencia']:.3f}")
        
        return metricas
    
    except Exception as e:
        logging.error(f"❌ Erro ao calcular métricas: {e}")
        return None

def gerar_relatorio_validacao(relatorio_qualidade, metricas, amostras, timestamp):
    """
    Gera relatório completo de validação
    """
    try:
        relatorio_completo = {
            'metadata': {
                'timestamp_geracao': datetime.now().isoformat(),
                'timestamp_processamento': timestamp,
                'versao_relatorio': '1.0'
            },
            'qualidade': relatorio_qualidade,
            'metricas': metricas,
            'amostras_validacao': amostras,
            'recomendacoes': []
        }
        
        # Gerar recomendações baseadas na análise
        recomendacoes = []
        
        if metricas and metricas['cobertura'] < 0.5:
            recomendacoes.append({
                'tipo': 'baixa_cobertura',
                'descricao': f"Cobertura de agrupamento baixa ({metricas['cobertura']:.1%}). Considere relaxar critérios de similaridade.",
                'prioridade': 'alta'
            })
        
        if metricas and metricas['densidade_media'] > 8:
            recomendacoes.append({
                'tipo': 'grupos_grandes',
                'descricao': f"Grupos muito grandes (média {metricas['densidade_media']:.1f}). Considere critérios mais rigorosos.",
                'prioridade': 'media'
            })
        
        if relatorio_qualidade and 'problemas_detectados' in relatorio_qualidade:
            for problema in relatorio_qualidade['problemas_detectados']:
                if problema['tipo'] == 'super_agrupamento':
                    recomendacoes.append({
                        'tipo': 'revisao_manual',
                        'descricao': f"Revisar manualmente grupos com >10 projetos: {problema['grupos_afetados']}",
                        'prioridade': 'alta'
                    })
        
        relatorio_completo['recomendacoes'] = recomendacoes
        
        # Salvar relatório
        arquivo_relatorio = f'resultados_agrupamento/relatorio_validacao_{timestamp}.json'
        with open(arquivo_relatorio, 'w', encoding='utf-8') as f:
            json.dump(relatorio_completo, f, indent=2, ensure_ascii=False, default=str)
        
        # Gerar versão resumida em texto
        arquivo_resumo = f'resultados_agrupamento/resumo_validacao_{timestamp}.txt'
        with open(arquivo_resumo, 'w', encoding='utf-8') as f:
            f.write("RELATÓRIO DE VALIDAÇÃO - AGRUPAMENTO DE PROJETOS\n")
            f.write("=" * 50 + "\n\n")
            
            if metricas:
                f.write("MÉTRICAS PRINCIPAIS:\n")
                f.write(f"• Cobertura: {metricas['cobertura']:.1%}\n")
                f.write(f"• Densidade média: {metricas['densidade_media']:.1f} projetos/grupo\n")
                f.write(f"• Eficiência: {metricas['eficiencia']:.3f}\n\n")
            
            if relatorio_qualidade:
                dist = relatorio_qualidade['distribuicao_grupos']
                f.write("DISTRIBUIÇÃO DOS GRUPOS:\n")
                f.write(f"• Total de grupos: {dist['total_grupos']}\n")
                f.write(f"• Tamanho médio: {dist['tamanho_medio']:.1f}\n")
                f.write(f"• Maior grupo: {dist['maior_grupo']} projetos\n\n")
            
            if recomendacoes:
                f.write("RECOMENDAÇÕES:\n")
                for i, rec in enumerate(recomendacoes, 1):
                    f.write(f"{i}. [{rec['prioridade'].upper()}] {rec['descricao']}\n")
        
        logging.info(f"📋 Relatório de validação salvo: {arquivo_relatorio}")
        logging.info(f"📄 Resumo salvo: {arquivo_resumo}")
        
        return {
            'relatorio_completo': arquivo_relatorio,
            'resumo': arquivo_resumo,
            'recomendacoes': recomendacoes
        }
    
    except Exception as e:
        logging.error(f"❌ Erro ao gerar relatório: {e}")
        return None

# Executar validação e qualidade
print("\n🔄 Executando Chunk 9: Validação e Qualidade")

if 'df_consolidado' in locals() and df_consolidado is not None and 'df_clean' in locals():
    print("🔍 Analisando qualidade dos agrupamentos...")
    
    # Análise de qualidade
    relatorio_qualidade = analisar_qualidade_grupos(df_consolidado, df_clean)
    
    # Cálculo de métricas
    metricas = calcular_metricas_agrupamento(df_consolidado)
    
    # Geração de amostras
    amostras = gerar_amostras_grupos(df_consolidado, df_clean)
    
    # Relatório de validação
    if 'timestamp_final' in locals():
        relatorio_final = gerar_relatorio_validacao(relatorio_qualidade, metricas, amostras, timestamp_final)
    else:
        timestamp_validacao = datetime.now().strftime('%Y%m%d_%H%M%S')
        relatorio_final = gerar_relatorio_validacao(relatorio_qualidade, metricas, amostras, timestamp_validacao)
    
    print(f"✅ Chunk 9 executado:")
    if metricas:
        print(f"   📈 Cobertura: {metricas['cobertura']:.1%}")
        print(f"   🎯 Densidade: {metricas['densidade_media']:.1f} proj/grupo")
    if relatorio_qualidade:
        print(f"   🏷️ Grupos válidos: {relatorio_qualidade['distribuicao_grupos']['total_grupos']}")
    print(f"   📋 Amostras geradas: {len(amostras)}")
    if relatorio_final and 'recomendacoes' in relatorio_final:
        print(f"   💡 Recomendações: {len(relatorio_final['recomendacoes'])}")
else:
    print("⚠️ Chunk 9 ignorado: Dados consolidados não disponíveis")